In [22]:
:dep caw = { path = "/Users/s/src/caw/caw", features = ["live", "widgets", "viz"] }

In [24]:
use caw::prelude::*;
let out: LiveStereoOut = 
    live_stereo_viz_udp(Default::default())
        .with_volume(knob("volume").build());

In [25]:
let knob_freq_hz: SV<f32> = sv(knob("freq_hz").build());

In [26]:
let lpf_freq_hz: SV<f32> = sv(knob("lpf_hz").build());

In [27]:
let clock_freq: SV<f32> = sv_default();

In [28]:
let notes: SV<Note> = sv_default();

In [29]:
let notes2: SV<Note> = sv_default();

In [30]:
let accents: SV<f32> = sv_default();

In [31]:
let clock: SV<bool> = sv(periodic_trig_hz(clock_freq.clone()).build());

In [32]:
let drums: SV<f32> = sv_default();

In [33]:
let baseline: SV<f32> = sv_default();

In [34]:
out.set_channel(|channel| {
    let offset = channel.circle_phase_offset_01();
    let freq = match channel {
        Channel::Left => sig_boxed(notes.clone().freq_hz()),
        Channel::Right => sig_boxed(notes.clone().freq_hz()),
    };
    let freq2 = match channel {
        Channel::Left => sig_boxed(notes2.clone().freq_hz()),
        Channel::Right => sig_boxed(notes2.clone().freq_hz()),
    };
    
    let sah = noise::white().filter(sample_and_hold(clock.clone()));
    let env = adsr(clock.clone().divide(1).trig_to_gate(0.01)).a(0.).r(0.5).build();
    (oscillator(Saw, freq).build() +
     oscillator(Saw, freq2).build())
        .filter(
            low_pass::default(
                10_000. * 
                lpf_freq_hz.clone() * 
                env * 
                (1. + sah * 0.2) * 
                accents.clone() *
                1.0
            ).q(1.))
        //.filter(down_sample(10.0))
        //.filter(quantizer(100.))
        .filter(compressor().scale(1.).ratio(0.2).threshold(0.5))
        .filter(chorus()
                .lfo_offset(ChorusLfoOffset::Interleave(channel))
                .lfo_rate_hz(0.1)
                .num_voices(1)
                .delay_s(0.001)
               )
         .filter(reverb().room_size(0.8))
        .filter(high_pass_butterworth(1.)) + drums.clone() + baseline.clone()
        
});

In [35]:
clock_freq.set(4.);

In [36]:
notes.set(note::C2);

In [37]:
notes.set(value_sequencer(clock.clone().divide(1), [
    note::C3,
    note::C5,
    note::A3,
    note::D4,
]));

In [38]:
notes2.set(value_sequencer(clock.clone().divide(1), [
    note::G3,
    note::F3,
    note::A3,
    note::D5,
    note::G3,
    note::F5,
    note::A3,
    note::D4,
    note::A5,
]));

In [39]:
accents.set(value_sequencer(clock.clone().divide(2), [
    0.2,
    0.2,
    0.4,
    0.2,
    1.0,
    0.2,
    0.2,
    0.2,
]));

In [57]:
use caw::viz_udp_app_lib::oscilloscope;
let mut viz2 = oscilloscope::Server::new("Drums", oscilloscope::Config {
    ..Default::default()
}).unwrap();

drums.set(
    (clock.clone().divide_with_offset(4, 0).trig(drum::kick()) * 0.5 +
    clock.clone().divide_with_offset(5, 2).trig(drum::snare()) * 0.5 +
        clock.clone().divide_with_offset(4, 1).trig(drum::snare()) * 0.5 +
    0.).filter(chorus()
                .lfo_rate_hz(0.1)
                .num_voices(1)
                .delay_s(0.001)
              )
    .for_each(move |x| {
        //println!("{}", x);
        let _ = viz2.send_samples_batched(&[x, 0.]);
    })
);


thread 'main' panicked at src/main.rs:77:22:
called `Result::unwrap()` on an `Err` value: Connection refused (os error 61)

Stack backtrace:
   0: std::backtrace::Backtrace::create
   1: anyhow::error::<impl core::convert::From<E> for anyhow::Error>::from
   2: caw_midi_udp_client::MidiUdpClient::send
   3: caw_midi_udp_widgets_app::main
   4: std::sys::backtrace::__rust_begin_short_backtrace
   5: std::rt::lang_start::{{closure}}
   6: std::rt::lang_start_internal
   7: _main

thread 'main' panicked at src/main.rs:77:22:
called `Result::unwrap()` on an `Err` value: Connection refused (os error 61)

Stack backtrace:
   0: std::backtrace::Backtrace::create
   1: anyhow::error::<impl core::convert::From<E> for anyhow::Error>::from
   2: caw_midi_udp_client::MidiUdpClient::send
   3: caw_midi_udp_widgets_app::main
   4: std::sys::backtrace::__rust_begin_short_backtrace
   5: std::rt::lang_start::{{closure}}
   6: std::rt::lang_start_internal
   7: _main

thread 'main' panicked at src/mai

In [41]:
baseline.set( {
    let env = adsr(clock.clone().divide(7).trig_to_gate(0.01)).a(0.).r(1.).build();
    super_saw(notes2.clone().freq_hz()).build()
        .filter(low_pass::default(6000. * env).q(1.0)) * 0.4
}
    );